In [5]:
from pathlib import Path
import pandas as pd
from skimage.feature import peak_local_max
import mrcfile
import napari
import napari_pyclesperanto_assistant
import starfile

In [33]:
eman2_dir = Path("/media/michalakdj/scratch/eman2_models/")
tomo_paths = [
    eman2_dir / Path("tomograms/ts001_corrected.mrc"),
    eman2_dir / Path("tomograms/ts002_corrected.mrc"),
    eman2_dir / Path("tomograms/ts003_corrected.mrc"),
    eman2_dir / Path("tomograms/ts004_corrected.mrc"),
    eman2_dir / Path("tomograms/ts005_corrected.mrc"),
]
tomo_names = [tomo_path.stem.split('_')[0] for tomo_path in tomo_paths]
segment_paths = [
    eman2_dir / Path("segmentations/ts001_corrected__ribosome_good_seg.mrc"),
    eman2_dir / Path("segmentations/ts002_corrected__ribosome_good_seg.mrc"),
    eman2_dir / Path("segmentations/ts003_corrected__ribosome_good_seg.mrc"),
    eman2_dir / Path("segmentations/ts004_corrected__ribosome_good_seg.mrc"),
    eman2_dir / Path("segmentations/ts005_corrected__ribosome_good_seg.mrc"),
]
tomo_bin_factor = 10
peak_separation = 12 # pixels
unbinned_pixel_size = 1.0825
print(f"Peaks will be at least {peak_separation * tomo_bin_factor * unbinned_pixel_size / 10.0} nm apart.")
threshold_rel = 0.1
viewer = napari_pyclesperanto_assistant.Viewer(ndisplay=3)

Peaks will be at least 12.99 nm apart.


In [34]:
# View the tomogram in Napari
tomos = [mrcfile.read(tomo) for tomo in tomo_paths]

name_i = 0
for tomo in tomos:
    viewer.add_image(
        tomo, 
        name=f"{tomo_names[name_i]}",
        depiction="plane", 
        blending="translucent", 
        visible=False,
        )
    name_i+=1

In [36]:
# Add the segmentation to the Napari viewer
segment_maps = [mrcfile.read(segment_path) for segment_path in segment_paths]

name_i = 0
for segment_map in segment_maps:
    viewer.add_image(
        segment_map, 
        name=f"{tomo_names[name_i]}_seg",
        colormap="gist_earth", 
        blending="translucent",
        visible=False, 
        )
    name_i+=1

In [37]:
colors = ["red","orange", "yellow","green", "blue", "purple"]

name_i = 0
peak_coords_dict = {}
for segment_map in segment_maps:
    peak_coords_dict[tomo_names[name_i]] = peak_local_max(
        segment_map, 
        min_distance=10, 
        threshold_rel=threshold_rel, 
        ) 
    viewer.add_points(
        peak_coords_dict[tomo_names[name_i]], 
        name=f"{tomo_names[name_i]} Peaks(rel_density > {threshold_rel})", 
        edge_color=colors[0], 
        edge_width=0.2, 
        blending="translucent",
        visible=False, 
        )
    name_i+=1

In [38]:
# Convert the calculated peak coordinates from binned space to unbinned space. Star files for RELION required unbinned coordinates.
rln_coordinates = ["rlnCoordinateX", "rlnCoordinateY", "rlnCoordinateZ"]
rln_angles = ["rlnAngleRot", "rlnAngleTilt", "rlnAnglePsi"]
rln_tomoname = "ts002"
peak_coords_df = pd.DataFrame(peak_coords,columns=rln_coordinates)
unbinned_peak_coords_df = pd.DataFrame()
for coordinate in rln_coordinates:
    unbinned_peak_coords_df[coordinate] = tomo_bin_factor * peak_coords_df[coordinate]
for angle in rln_angles:
    unbinned_peak_coords_df[angle] = 0.0
unbinned_peak_coords_df["rlnTomoName"] = rln_tomoname
unbinned_peak_coords_df.head(3)

# Create empty pd columns and append entries for every tomogram in tomos
name_i = 0
unbinned_peak_coords_dict = {}
for tomo in peak_coords_dict:
    peak_coords_dict[tomo]= pd.DataFrame(peak_coords_dict[tomo], columns=rln_coordinates)
    unbinned_peak_coords_dict[tomo] = peak_coords_dict[tomo] * tomo_bin_factor
    for angle in rln_angles:
        unbinned_peak_coords_dict[tomo][angle] = 0.0
    unbinned_peak_coords_dict[tomo]["rlnTomoName"] = tomo_names[name_i]
    name_i+=1

In [44]:
unbinned_peak_coords_df = pd.concat(unbinned_peak_coords_dict.values(), ignore_index=True)
unbinned_peak_coords_df

,rlnCoordinateX,rlnCoordinateY,rlnCoordinateZ,rlnAngleRot,rlnAngleTilt,rlnAnglePsi,rlnTomoName
0,2160,3780,4590,0.0,0.0,0.0,ts001
1,2510,8440,4250,0.0,0.0,0.0,ts001
2,1920,2980,5370,0.0,0.0,0.0,ts001
3,1850,3760,4570,0.0,0.0,0.0,ts001
4,2440,9020,5010,0.0,0.0,0.0,ts001
...,...,...,...,...,...,...,...
1907,1930,3980,4390,0.0,0.0,0.0,ts005
1908,2870,8020,3310,0.0,0.0,0.0,ts005
1909,1730,2000,7930,0.0,0.0,0.0,ts005
1910,2100,7680,7450,0.0,0.0,0.0,ts005


In [45]:
write_path = Path("/media/michalakdj/scratch/eman2_models/segmentations/ribosomes.star")
starfile.write(unbinned_peak_coords_df, write_path, overwrite=True)